In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Randomly seeding
tf.random.set_seed(6950)

# Biosppy for signal processing
from biosppy.signals import ecg

# Constants
SAMPLING_RATE = 500  # Hz
ecg_folder = "../../../Datasets/12-lead electrocardiogram database/ECGData"
diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}


# Feature extraction function
def extract_comprehensive_features(ecg_signal):
    """
    Extract comprehensive features from entire ECG signal
    """
    try:
        ecg_processed = ecg.ecg(signal=ecg_signal, sampling_rate=SAMPLING_RATE, show=False)
        rpeaks = ecg_processed['rpeaks']
        heart_rate = ecg_processed['heart_rate']

        features = [
            np.mean(ecg_signal),
            np.std(ecg_signal),
            np.median(ecg_signal),
            np.min(ecg_signal),
            np.max(ecg_signal),
            len(rpeaks),
            np.mean(heart_rate) if len(heart_rate) > 0 else 0,
            np.std(heart_rate) if len(heart_rate) > 0 else 0,
            np.mean(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.std(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.percentile(ecg_signal, 25),
            np.percentile(ecg_signal, 75),
        ]

        return features

    except Exception as e:
        print(f"Error processing signal: {e}")
        return [0] * 12


# Dataset preparation function
def prepare_dataset(ecg_folder, diagnostics_df):
    signals, signal_features, signal_labels = [], [], []

    for _, row in diagnostics_df.iterrows():
        file_name = row['FileName']
        rhythm_label = row['Rhythm']

        if pd.isnull(rhythm_label) or rhythm_label not in rhythm_mapping.values():
            continue

        ecg_file = os.path.join(ecg_folder, f"{file_name}.csv")
        if not os.path.exists(ecg_file):
            continue

        ecg_data = pd.read_csv(ecg_file, header=0).iloc[:, 1].values
        ecg_data = ecg_data.astype(float)

        features = extract_comprehensive_features(ecg_data)
        signals.append(ecg_data)
        signal_features.append(features)
        signal_labels.append(rhythm_label)

    return np.array(signals), np.array(signal_features), np.array(signal_labels)


# Load and process data
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)

raw_signals, signal_features, signal_labels = prepare_dataset(ecg_folder, diagnostics_df)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(signal_labels)
onehot_labels = to_categorical(encoded_labels)


# Create MLP model
def create_mlp_model(input_shape, num_classes):
    mlp = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return mlp


# Stratified K-Fold Cross-Validation with optimizer tuning
optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
results = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for optimizer in optimizers:
    print(f"\nTesting optimizer: {optimizer}")
    cv_scores = []

    for train_index, test_index in skf.split(signal_features, encoded_labels):
        X_train, X_test = signal_features[train_index], signal_features[test_index]
        y_train, y_test = onehot_labels[train_index], onehot_labels[test_index]

        mlp = create_mlp_model(input_shape=X_train.shape[1], num_classes=onehot_labels.shape[1])
        mlp.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        history = mlp.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=1)
        _, accuracy = mlp.evaluate(X_test, y_test, verbose=0)
        cv_scores.append(accuracy)

        y_pred = mlp.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test, axis=1)

        print("\nClassification Report:")
        print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_, digits=5))

    results[optimizer] = np.mean(cv_scores)
    print(f"Optimizer: {optimizer} - Mean CV Accuracy: {np.mean(cv_scores)}")

print("\nFinal Results:")
for opt, acc in results.items():
    print(f"{opt}: {acc}")

2024-12-01 15:07:07.751490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 15:07:07.813070: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 15:07:07.830383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 15:07:07.942142: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 15:07:08.682660: W tensorflow/compiler/tf2


Testing optimizer: adam


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1733044411.810282  127638 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733044411.939601  127638 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733044411.

Epoch 1/100


I0000 00:00:1733044412.815090  131412 service.cc:146] XLA service 0x7cebd80036d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733044412.815117  131412 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-12-01 15:13:32.851651: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-01 15:13:33.000095: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


...

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.3107 - loss: 37.8056 - val_accuracy: 0.4877 - val_loss: 6.2006
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - accuracy: 0.3366 - loss: 21.3227 - val_accuracy: 0.5469 - val_loss: 3.3583
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.3521 - loss: 16.9509 - val_accuracy: 0.5387 - val_loss: 2.7144
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.3698 - loss: 13.9087 - val_accuracy: 0.5076 - val_loss: 2.8312
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.3465 - loss: 12.7276 - val_accuracy: 0.4536 - val_loss: 3.3796
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - accuracy: 0.3691 - loss: 11.3253 - val_accuracy: 0.4683 - val_loss: 3.0488
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step - accuracy: 0.3463 - loss: 10.5598 - val_accuracy: 0.3914 - val_loss: 3.7920
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step - accuracy: 0.3698 - l

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2135 - loss: 106.0002 - val_accuracy: 0.3016 - val_loss: 28.2313
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.3205 - loss: 33.9416 - val_accuracy: 0.3122 - val_loss: 18.3635
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - accuracy: 0.3460 - loss: 27.9284 - val_accuracy: 0.3016 - val_loss: 13.0766
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3725 - loss: 23.1321 - val_accuracy: 0.3104 - val_loss: 9.0473
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.3674 - loss: 20.3868 - val_accuracy: 0.2887 - val_loss: 7.7330
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.3811 - loss: 18.5392 - val_accuracy: 0.2852 - val_loss: 6.6136
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3788 - loss: 16.9590 - val_accuracy: 0.3257 - val_loss: 5.7887
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - accuracy: 0.4013 - loss: 15.4758

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.3152 - loss: 72.8329 - val_accuracy: 0.4536 - val_loss: 8.7504
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - accuracy: 0.3648 - loss: 37.1650 - val_accuracy: 0.4871 - val_loss: 7.7982
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - accuracy: 0.3672 - loss: 29.6754 - val_accuracy: 0.5123 - val_loss: 7.8352
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - accuracy: 0.3896 - loss: 25.1386 - val_accuracy: 0.4953 - val_loss: 7.3616
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - accuracy: 0.3864 - loss: 22.7075 - val_accuracy: 0.4296 - val_loss: 7.8428
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.3976 - loss: 20.3764 - val_accuracy: 0.4243 - val_loss: 7.5212
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - accuracy: 0.4000 - loss: 18.9100 - val_accuracy: 0.4437 - val_loss: 6.7167
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - accuracy: 0.4179 - loss: 16.8971